This notebook is intended to run the full LandTrendr Optimization (LTOP) workflow implemented in Python. This tool is intended to be used to select the 'optiumum' version of the LandTrendr change detection algorithm (Kennedy et al., 2010, 2018) for different areas on a landscape. 

#### Notes
- You need to authenticate the run with a specific GEE account. In the import params statement its going to trigger the ee.Authenticate() protocal which will prompt you to a browser page to authenticate using your GEE account. 
- Subsequent scripts have just the ee.Initialize protocal because you should have already authenicated and this should only happen once. 
- Right now this is going to import the params from a separate .py file and treat those as imports. It may actually be easier/more straight forward if we're going to stick with a Jupyter Notebook to just put these directly into a cell here in the Jupyter Notebook. 
- If you import a module at the top and then change something in the exe script you will need to restart or otherwise delete variables because that change won't be reflected otherwise. 

In [2]:
import ee 
import params
import time 
import ltop 
from run_SNIC_01 import RunSNIC
import run_kMeans_02_1 as kmeans_1
# import run_kMeans_02_2 as kmeans_2

There is potentially an issue here where running a cell for a large area will result in a problem with the cell hanging. This may be an issue particuarly if you're running this thing in a browser. Its also possible that it will just start a job on the GEE server and then finish running here in the notebook. 

In order to check if a task is done we can get the task status and check to see if its done. Define a little function that can be recycled below for that purpose. 

In [3]:
def check_task_status(task_dict): 
    '''
    Input to this function should be a dictionary that is formatted 
    like the output of task.status()
    '''
    task_id = task_dict['id']
    #for some reason GEE defaults this to a list with a dictioanary as its only item
    task_status = ee.data.getTaskStatus(task_id)[0]
    
    return task_status['state']

In [4]:
#run the first SNIC step 

snic = RunSNIC(params)

status1,status2 = snic.generate_tasks()


In [5]:
#run the second kmeans step, checking first to see if the first one is done
# snic_pts

# snic_pts = ee.FeatureCollection(snic_pts).first()
# snic_pts
task_status = check_task_status(status1)
while task_status: 
    if task_status == 'COMPLETED': 
        print('The previous task is complete')
        km_status = kmeans_1.generate_tasks()
        break
    